A notebook demostrating the use of Archie's Law to estimate water qulity estimates. The equation is the following.

logCw = logCt – mlogØ

 

where Cw is the predicted fluid EC (S/m), Ct is bulk conductivity and logØ is the free water.

We are trying to estimate fluid EC at on a grid given AEM conductivity (interpolated) and mobile water (interpolated) and a cementation factor based on empirical relationships developed using pore fluid EC and borehole NMR.

Neil Symington
neil.symington@ga.gov.au

In [1]:
%matplotlib inline

In [2]:
import pandas
import rasterio
import numpy as np
from hydrogeol_utils import SNMR_utils, AEM_utils, spatial_functions, borehole_utils
import netCDF4
import math
import os, glob
import gc
from geophys_utils._netcdf_line_utils import NetCDFLineUtils
from geophys_utils._netcdf_point_utils import NetCDFPointUtils
import rasterio
from rasterio import Affine
from rasterio.warp import reproject, Resampling
from scipy import interpolate
import matplotlib.pyplot as plt

In [3]:
def resample_raster2array(infile, newaff, new_shape, driver='GTiff',
                    null = -999):

    # Open
    src = rasterio.open(infile)

    # Extract data as an array
    arr = src.read()[0]

    # Get the affine
    aff = src.transform


    # Create new array with the grid
    # coordinates from the kriging
    newarr = np.empty(shape=new_shape)

    # Reproject
    reproject(
        arr, newarr,
        src_transform=aff,
        dst_transform=newaff,
        src_crs=src.crs,
        dst_crs=src.crs,
        resampling=Resampling.bilinear)

    src.close()

    # Do some post processing
    #newarr[np.abs(newarr - null) < 0.000001] = np.nan
    newarr[newarr < -900] = np.nan

    return newarr

def slice_conductivity(logcondprofile, cond_intervals, base, top):
    old_intervals = cond_intervals.copy()
    
    old_intervals[:,0]= old_intervals[:,0] + 0.01

    old_intervals = np.sort(old_intervals.flatten())
    
    # repeat params
    params = np.repeat(logcondprofile, 2)
    
    # Interpolate

    f = interpolate.interp1d(old_intervals,
                             params, kind='linear')
    
    new_intervals = np.array([top,base])
    
    xnew = new_intervals
    
    xnew = xnew[(xnew > np.min(old_intervals)) & (xnew < np.max(old_intervals))]

    xnew = np.sort(np.concatenate((old_intervals, xnew)))

    ynew = f(xnew)
    mask = np.logical_and(xnew <= base, xnew >= top)
    
    new_profile = ynew[mask]
    new_depths = xnew[mask]
    
    # Calculate thickness
    thickness = spatial_functions.depth_to_thickness(new_depths)[:-1]
    # Calculate weights based on thickness
    weights = thickness/(thickness.sum())
    
    # Now multiply these weights by the profile
    weighted_logcond = weights * new_profile[:-1]
    
    return(np.sum(weighted_logcond))
    


In [4]:
# We also want to reshape the mobile water raster

#infile = r"D:\GA\aquifer_properties\interpolated\cenozoic_mobile.tif"
infile = r"D:\GA\aquifer_properties\interpolated\permocarb_mobile.tif"

mob_water_src = rasterio.open(infile)

RasterioIOError: 'D:\GA\aquifer_properties\interpolated\permocarb_mobile.tif' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
arr = mob_water_src.read()[0]

# Get the affine
aff = mob_water_src.transform

# GEt log of the water content

logMob = np.log10(arr)

In [ ]:
plt.close()
plt.imshow(arr)
plt.show()

In [ ]:
# OPen an AEM grid, resample them onto the same affine as the stratigraphy
# and save them as a multiband array

import glob

os.chdir(r"D:\GA\EK_AEM\OrdKeep_borehole_constrained\gdal_grids\grids\layergrids")

cond = np.nan*np.ones(shape = (30, arr.shape[0], arr.shape[1]),
                      dtype = np.float32)
intervals = np.zeros(shape = (30,2), dtype = np.float32)

for i, file in enumerate(glob.glob('*.tif')):
    
    print(file)
    
    cond[i] = resample_raster2array(file, aff, arr.shape,
                                    driver='GTiff', null = 0)
    
    intervals[i] = [float(file.split('_')[2]), float(file.split('_')[3])]

In [ ]:
cond_intervals = intervals.copy()

In [ ]:
# Create the log conductivity array

logcond =np.log10(cond)


In [ ]:
plt.close()
plt.imshow(logcond[0])

plt.show()

In [ ]:
# Open
#infile = r"D:\GA\KeepWaterTable\raster_elevation_interpolation\nulled\KeepRiver_water_table_mBGL.tif"
infile = r"D:\GA\AEM_interp_OrdKeep\Neil\elevatoin\Cuk_Pk_top.tif"

strat_top = resample_raster2array(infile, aff, logMob.shape,
                                    driver='GTiff', null = -3.40282e+38)

#infile = r"D:\GA\AEM_interp_OrdKeep\Neil\elevatoin\Cz_base.tif"
infile = r"D:\GA\AEM_interp_OrdKeep\Neil\elevatoin\Cuk_Pk_base.tif"

strat_base = resample_raster2array(infile, aff, logMob.shape,
                                    driver='GTiff', null = -3.40282e+38
                                      )


In [ ]:
# We need to conver the strat top and base to depth

# Bring in the LiDAR

infile = r"D:\GA\rasters\EK_LiDAR_cubic_5x5.tif"

lid5m = resample_raster2array(infile, aff, logMob.shape,
                                    driver='GTiff', null = -9999.
                                      )

In [ ]:
# Now convert strat top and base to depth by subtracting them from the LiDAR

#strat_top_depth = strat_top
strat_top_depth = lid5m - strat_top
strat_base_depth = lid5m - strat_base

In [ ]:
# Do some post processing for negative depths

strat_top_depth[strat_top_depth < 0] = 0

In [ ]:
depth_top = intervals[:,0]

In [ ]:
# Now we want to iterate through the strat coordinates
# find top and bottom and use this to find the AEM conductivity

log_AEM_interval = np.nan*np.ones(shape = (strat_top_depth.shape), dtype = np.float32)

for i in range(strat_top_depth.shape[0]):
    for j in range(strat_top_depth.shape[1]):
        top = strat_top_depth[i,j]
        base = strat_base_depth[i,j]

        if (~np.isnan(top)) and (top < base):
            top_ind = np.searchsorted(depth_top, top)
            base_ind = np.searchsorted(depth_top, base)
            
            # Now use these indices to slice and average the log10 AEM
            if top_ind == base_ind:
                log_AEM_interval[i,j] = logcond[top_ind,i,j]
            elif top_ind < base_ind:
                log_AEM_interval[i,j] = slice_conductivity(logcond[:,i,j], cond_intervals, base, top)
            else:
                print('error')


        


In [ ]:
# Now we can estimate our water quality value

#m = 0.509 # From KP
m = 0.726

logEC = log_AEM_interval - (m * logMob)

In [ ]:
# Convert to EC

EC = 10**logEC

In [ ]:
# Export as a raster

outfile = r"D:\GA\waterQuality\permocarb_EC.tif"
#outfile = r"D:\GA\waterQuality\Cenozoic_EC.tif"

new_dataset = rasterio.open(outfile, 'w', driver='GTiff',
                                height= EC.shape[0], width=EC.shape[1],
                                count=1, dtype=np.float32,
                                crs=mob_water_src.crs, transform=aff)
new_dataset.write(EC, 1)

new_dataset.close()

In [ ]:
TDS = EC/1.57 * 1000. # From Steve

In [ ]:
# Export as a raster

outfile = r"D:\GA\waterQuality\permocarb_TDS.tif"
#outfile = r"D:\GA\waterQuality\Cenozoic_TDS.tif"

new_dataset = rasterio.open(outfile, 'w', driver='GTiff',
                                height= TDS.shape[0], width=TDS.shape[1],
                                count=1, dtype=np.float32,
                                crs=mob_water_src.crs, transform=aff)
new_dataset.write(TDS, 1)

new_dataset.close()